In [2]:
import pandas as pd
pd.options.display.max_columns = 999
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import json
import time
import random
import datetime as dt
from datetime import timedelta
import itertools

from sklearn import ensemble
from sklearn import linear_model
from sklearn import svm
from sklearn.linear_model import RandomizedLasso
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
import sklearn.cross_validation
from sklearn import discriminant_analysis
from sklearn import naive_bayes
from sklearn import tree
from sklearn import cluster
from sklearn import metrics
from sklearn import cross_validation
from sklearn import svm

import xgboost as xgb

from os.path import expanduser
home = expanduser("~")
path = home + "/Dropbox/python/"

In [3]:
from IPython.core.display import HTML
def css_styling():
    styles = open(path + "custom.css", "r").read()
    return HTML(styles)
css_styling()

In [4]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [5]:
iris = pd.read_csv(path + "iris.data", header=None)
y = iris[4]
iris = iris.drop([4], 1)
# x_train, x_test, y_train, y_test = sklearn.cross_validation.train_test_split(iris, y, test_size = .20, random_state=42)

In [6]:
df = pd.read_csv(path + "Oracle Development/winequality-red.csv", sep=';')
Y = df['quality'].values
df = df.drop('quality',1)
# x_train, x_test, y_train, y_test = sklearn.cross_validation.train_test_split(df, Y, test_size = .20, random_state=42)

In [7]:
cancer = pd.read_csv(path + 'Oracle Development/breast_cancer.csv')
#http://www.is.umk.pl/projects/datasets.html
y = np.array(cancer['2.1'])
x = np.array(cancer.drop(['2.1', '1.3'], 1))
# x_train, x_test, y_train, y_test = sklearn.cross_validation.train_test_split(x, y, test_size = .20, random_state=42)

In [8]:
votes = pd.read_csv(path + '/Oracle Development/votes.csv')
votes = votes.replace('?', float('NaN'))
votes = votes.dropna()
votes = votes.reset_index(drop=True)
votes['0'] = votes.replace('D', 1)
votes['0'] = votes.replace('R', 0)
y = np.array(votes['0'])
votes = votes.drop('0', axis = 1)
votes = votes.replace('y', 1)
votes = votes.replace('n', 0)
x = np.array(votes)
# x_train, x_test, y_train, y_test = sklearn.cross_validation.train_test_split(np.array(x), np.array(y), test_size = .20, random_state=42)

In [10]:
leads = pd.read_csv(path + 'Movoto/leads.csv')
one_hot_columns = ['First_Lead_Platform', 'State']
for i in one_hot_columns:
    dummies = pd.get_dummies(leads[i])
    for j in dummies:
        leads[j] = dummies[j]
y = leads['Met']
leads = leads.drop(['First_Lead_Platform', 'State', 'Met'], axis =1)

In [12]:
def evaluate(hyperparameters='individual'):
    iris = pd.read_csv(path + "iris.data", header=None)
    y = iris[4]
    iris = iris.drop([4], 1)
    print "Iris:"
    
    problem = classifier(iris,y, hyperparameters=hyperparameters)
    problem.predict()
    
    
    
    df = pd.read_csv(path + 'Oracle Development/winequality-red.csv', sep=';')
    Y = df['quality'].values
    df = df.drop('quality',1)
    print"\nWine Quality:"
    
    problem = classifier(df, Y, hyperparameters=hyperparameters)
    problem.predict()
    
    
    
    cancer = pd.read_csv(path + 'Oracle Development/breast_cancer.csv')
    #http://www.is.umk.pl/projects/datasets.html
    y = np.array(cancer['2.1'])
    x = np.array(cancer.drop(['2.1', '1.3'], 1))
    print "\nBreast Cancer:"
    
    problem = classifier(x, y, hyperparameters=hyperparameters)
    problem.predict()
    
    
    
    votes = pd.read_csv(path + '/Oracle Development/votes.csv')
    votes = votes.replace('?', float('NaN'))
    votes = votes.dropna()
    votes = votes.reset_index(drop=True)
    votes['0'] = votes.replace('D', 1)
    votes['0'] = votes.replace('R', 0)
    y = np.array(votes['0'])
    votes = votes.drop('0', axis = 1)
    votes = votes.replace('y', 1)
    votes = votes.replace('n', 0)
    x = np.array(votes)
    print "\nVotes:"
    
    problem = classifier(x, y, hyperparameters=hyperparameters)
    problem.predict()


In [301]:
class classifier():
    def __init__(self, x, y, test=None, hyperparameters='individual'):
        self.time = time.time()
        self.hyperparameters = hyperparameters
        if type(y[0]) != int:
            y, self.mapping = self.categorize(y)
        self.x = np.array(x)
        self.y = np.array(y, dtype = int)
        if test is None:
            self.test = None
        else:
            self.test = np.array(test)            
        self.x_train, self.x_test, self.y_train, self.y_test = sklearn.cross_validation.train_test_split\
            (self.x, self.y, test_size = .20, random_state=42)
        self.x_train1 = self.x_train[:len(self.x_train)/2]
        self.x_train2 = self.x_train[len(self.x_train)/2:]
        self.y_train1 = self.y_train[:len(self.x_train)/2]
        self.y_train2 = self.y_train[len(self.x_train)/2:]
        self.stacking_train = self.x_train2
        self.stacking_test = self.x_test
        
        self.x1 = self.x[:len(self.x)/2]
        self.x2 = self.x[len(self.x)/2:]
        self.y1 = self.y[:len(self.x)/2]
        self.y2 = self.y[len(self.x)/2:]
        self.test_stacking_train = self.x2
        self.test_stacking_test = self.test
        
        self.stacking = False
        self.testing = False
        self.optimal_iterator = None
        
        self.algorithms = [self.xgboost, self.gradient_booster, self.random_forest,\
              self.extremely_random_forest, self.logistic_regression, self.lda, self.qda]
        
        self.xgb_max_depth = None
        self.gb_n_estimators = None
        self.gb_max_depth = None
        self.rf_n_estimators = None
        self.rf_max_features = None
        self.erf_n_estimators = None
        self.erf_max_features = None
        self.log_reg_penalty = None
        self.log_reg_C = None
        self.qda_reg_param = None
        self.svc_C = None
        self.svc_kernel = None
        
        self.training_df = None
        self.testing_df = None
        
        
    
    def accuracy(self, predictions):
        if len(predictions) == len(self.y_test):
            y_test = self.y_test
        elif len(predictions) == len(self.y_train2):
            y_test = self.y_train2
        elif len(predictions) == len(self.y2):
            y_test = self.y2
        else:
            print "Accuracy called with incorrectly sized predictions"
        count = 0
        for i in xrange(len(predictions)):
            if predictions[i] == y_test[i]:
                count += 1
        return count/float(len(predictions))
    
    def categorize(self, labels):
        mapping = {}
        labels = pd.Series(labels)
        for i,v in enumerate(list(set(labels))):
            labels = labels.replace(v, i)
            mapping[i] = v
        return labels, mapping  
    
    def stacker(self):
        self.stacking = True
        self.train_stacker()
        self.stacking = False
        return self.test_stacker()
    
    def train_stacker(self):
        for model in self.algorithms:
            generate_train, generate_test = model()
            if self.testing == False:
                self.stacking_train = np.column_stack((self.stacking_train, generate_train))
                self.stacking_test = np.column_stack((self.stacking_test, generate_test))
            else:
                self.test_stacking_train = np.column_stack((self.test_stacking_train, generate_train))
                self.test_stacking_test = np.column_stack((self.test_stacking_test, generate_test))
        return
        
    def test_stacker(self):
        if self.testing == False:
            return [model(self.stacking_train, self.y_train2, self.stacking_test, self.y_test) \
                for model in self.algorithms]
        else:
            return [model(self.test_stacking_train, self.y2, self.test_stacking_test, None)\
                for model in self.algorithms]
            
    def initialize_data(self, x_train=None, y_train=None, x_test=None, y_test=None):
        if self.testing == False:
            if self.stacking == False:
                return self.x_train, self.y_train, self.x_test, self.y_test
            else:
                if x_train == None:
                    x_train = self.x_train1
                if y_train == None:
                    y_train = self.y_train1
                if x_test == None:
                    x_test = self.x_train2
                if y_test == None:
                    y_test = self.y_train2
                return x_train, y_train, x_test, y_test
        else:
            if self.stacking == False:
                return self.x, self.y, self.test, None
            else:
                if x_train == None:
                    x_train = self.x1
                if y_train == None:
                    y_train = self.y1
                if x_test == None:
                    x_test = self.x2
                if y_test == None:
                    y_test = self.y2
                return x_train, y_train, x_test, y_test
    
    def stacking_test_data(self):
        if self.testing == False:
            return self.x_test
        else:
            return self.test
    
    def xgboost(self, x_train=None, y_train=None, x_test=None, y_test=None):
        x_train, y_train, x_test, y_test = self.initialize_data(x_train, y_train, x_test, y_test)
        dtrain = xgb.DMatrix(x_train, label=y_train)
        dtest = xgb.DMatrix(x_test)
        param = {'bst:max_depth':50, 'bst:eta':1, 'silent':1,'objective':'multi:softmax'}
        param['num_class'] = x_train.shape[1]
#         evallist = [(dtest,'eval'), (dtrain,'train')]
        num_round = 10
        bst = xgb.train(param, dtrain, num_round, verbose_eval=False)
        preds = bst.predict(dtest)
#         for i in xrange(len(labels)):
#             print labels[i], self.y_test[i]
        if self.stacking == True:
            return preds, bst.predict(xgb.DMatrix(self.stacking_test_data()))
        return preds
    
    def gradient_booster(self, x_train=None, y_train=None, x_test=None, y_test=None):
        x_train, y_train, x_test, y_test = self.initialize_data(x_train, y_train, x_test, y_test)
        if self.hyperparameters == 'default':
            booster = ensemble.GradientBoostingClassifier(n_estimators = 400, max_depth = 2)
            booster = booster.fit(x_train, y_train)
            booster_predictions = booster.predict(x_test)
            if self.stacking == True:
                return booster_predictions, booster.predict(self.stacking_test_data())
            return booster_predictions
        elif self.hyperparameters == 'individual':
            if self.gb_n_estimators == None:
                max_accuracy = -1
                for n_estimators in xrange(100, 1300, 300):
                    booster = ensemble.GradientBoostingClassifier\
                    (n_estimators = n_estimators)
                    booster = booster.fit(x_train, y_train)
                    booster_predictions = booster.predict(x_test)
                    if self.accuracy(booster_predictions) > max_accuracy:
                        max_accuracy = self.accuracy(booster_predictions)
                        self.gb_n_estimators = n_estimators
            if self.gb_max_depth == None:
                max_accuracy = -1
                for max_depth in xrange(2,15,3):
                    booster = ensemble.GradientBoostingClassifier\
                    (max_depth = max_depth)
                    booster = booster.fit(x_train, y_train)
                    booster_predictions = booster.predict(x_test)
                    if self.accuracy(booster_predictions) > max_accuracy:
                        max_accuracy = self.accuracy(booster_predictions)
                        self.gb_max_depth = max_depth
                print "Optimal Booster Hyperparams: n_estimators = %d, max_depth = %d" \
                        %(self.gb_n_estimators, self.gb_max_depth)
            booster = ensemble.GradientBoostingClassifier(\
                          n_estimators = self.gb_n_estimators,
                          max_depth = self.gb_max_depth)
            booster = booster.fit(x_train, y_train)
            booster_predictions = booster.predict(x_test)
            if self.stacking == True:
                return booster_predictions, booster.predict(self.stacking_test_data())
            return booster_predictions
        elif self.hyperparameters == 'grid':
            max_accuracy = -1
            for n_estimators in xrange(100, 1000, 300):
                for max_depth in xrange(2,15,3):
                    booster = ensemble.GradientBoostingClassifier\
                    (n_estimators = n_estimators, max_depth = max_depth)
                    booster = booster.fit(x_train, y_train)
                    booster_predictions = booster.predict(x_test)
                    print str(self.accuracy(booster_predictions)) + ' ' + str(n_estimators) + ' ' + str(max_depth)
                    if self.accuracy(booster_predictions) > max_accuracy:
                        max_accuracy = self.accuracy(booster_predictions)
                        max_predictions = booster_predictions
                        optimal_parameters = [n_estimators, max_depth]
            print("Gradient booster: ", optimal_parameters)
            return max_predictions, 
                
    
    def random_forest(self, x_train=None, y_train=None, x_test=None, y_test=None):
        x_train, y_train, x_test, y_test = self.initialize_data(x_train, y_train, x_test, y_test)
        if self.hyperparameters == 'default':
            forest = ensemble.RandomForestClassifier(n_estimators=100)
            forest = forest.fit(x_train, y_train)
            forest_predictions = forest.predict(x_test)
            return forest_predictions
        elif self.hyperparameters == 'individual':
            if self.rf_n_estimators == None:
                max_accuracy = -1
                for n_estimators in xrange(100, 1400, 400):
                    forest = ensemble.RandomForestClassifier(n_estimators = n_estimators)
                    forest = forest.fit(x_train, y_train)
                    forest_predictions = forest.predict(x_test)
                    if self.accuracy(forest_predictions) > max_accuracy:
                        max_accuracy = self.accuracy(forest_predictions)
                        self.rf_n_estimators = n_estimators
            if self.rf_max_features == None:
                feat = x_train.shape[1]
                max_accuracy = -1
                for max_features in xrange(int(.3*feat), int(.6*feat), int(.05*feat)+1):
                    forest = ensemble.RandomForestClassifier(max_features = max_features)
                    forest = forest.fit(x_train, y_train)
                    forest_predictions = forest.predict(x_test)
                    if self.accuracy(forest_predictions) > max_accuracy:
                        max_accuracy = self.accuracy(forest_predictions)
                        self.rf_max_features = max_features
                print "Optimal Forest Hyperparams: n_estimators = %d, max_features = %d" \
                        %(self.rf_n_estimators, self.rf_max_features)
            forest = ensemble.RandomForestClassifier(\
                 n_estimators=self.rf_n_estimators, max_features = self.rf_max_features)
            forest = forest.fit(x_train, y_train)
            forest_predictions = forest.predict(x_test)
            if self.stacking == True:
                return forest_predictions, forest.predict(self.stacking_test_data())
            return forest_predictions 
        elif self.hyperparameters == 'grid':
            feat = x_train.shape[1]
            max_accuracy = -1
            for n_estimators in xrange(100, 1000, 200):
                for max_features in xrange(int(.3*feat), int(.6*feat), int(.03*feat)+1):
                    for criterion in ['gini', 'entropy']:
                        forest = ensemble.RandomForestClassifier(n_estimators = \
                                 n_estimators, criterion = criterion, max_features = max_features)
                        forest = forest.fit(x_train, y_train)
                        forest_predictions = forest.predict(x_test)
                        print self.accuracy(forest_predictions)
                        if self.accuracy(forest_predictions) > max_accuracy:
                            max_accuracy = self.accuracy(forest_predictions)
                            max_predictions = forest_predictions
                            optimal_parameters = [n_estimators, max_features, criterion]
            print["Random forest: ", optimal_parameters]
            return max_predictions
                    
    
    def extremely_random_forest(self, x_train=None, y_train=None, x_test=None, y_test=None):
        x_train, y_train, x_test, y_test = self.initialize_data(x_train, y_train, x_test, y_test)
        if self.hyperparameters == 'default':
            erf = ensemble.ExtraTreesClassifier(n_estimators=100)
            erf = erf.fit(x_train, y_train)
            erf_predictions = erf.predict(x_test)
            return erf_predictions
        elif self.hyperparameters == 'individual':
            if self.erf_n_estimators == None:
                max_accuracy = -1
                for n_estimators in xrange(100, 1300, 400):
                    erf = ensemble.ExtraTreesClassifier(n_estimators = n_estimators)
                    erf = erf.fit(x_train, y_train)
                    erf_predictions = erf.predict(x_test)
                    if self.accuracy(erf_predictions) > max_accuracy:
                        max_accuracy = self.accuracy(erf_predictions)
                        self.erf_n_estimators = n_estimators
            if self.erf_max_features == None:
                feat = x_train.shape[1]
                max_accuracy = -1
                for max_features in xrange(int(.3*feat), int(.6*feat), int(.05*feat)+1):
                    erf = ensemble.ExtraTreesClassifier(max_features = max_features)
                    erf = erf.fit(x_train, y_train)
                    erf_predictions = erf.predict(x_test)
                    if self.accuracy(erf_predictions) > max_accuracy:
                        max_accuracy = self.accuracy(erf_predictions)
                        self.erf_max_features = max_features
                print "Optimal erf Hyperparams: n_estimators = %d, max_features = %d" \
                        %(self.erf_n_estimators, self.erf_max_features)
            erf = ensemble.ExtraTreesClassifier(\
                 n_estimators=self.erf_n_estimators, max_features = self.erf_max_features)
            erf = erf.fit(x_train, y_train)
            erf_predictions = erf.predict(x_test)
            if self.stacking == True:
                return erf_predictions, erf.predict(self.stacking_test_data())
            return erf_predictions
    
    def logistic_regression(self, x_train=None, y_train=None, x_test=None, y_test=None):
        x_train, y_train, x_test, y_test = self.initialize_data(x_train, y_train, x_test, y_test)
        if self.hyperparameters == 'default':
            logistic = linear_model.LogisticRegression(penalty="l2", C=1)#class_weight='auto'
            logistic.fit(x_train, y_train)
            logistic_predictions = logistic.predict(x_test)
            if self.stacking == True:
                return logistic_predictions, logistic.predict(self.stacking_test_data())
            return logistic_predictions
        elif self.hyperparameters == 'individual':
            if self.log_reg_penalty == None:
                for penalty in ['l1', 'l2']:
                    if penalty == 'l1':
                        logistic = linear_model.LogisticRegression(penalty="l1")#class_weight='auto'
                        logistic.fit(x_train, y_train)
                        logistic_predictions = logistic.predict(x_test)
                        l1_accuracy = self.accuracy(logistic_predictions)
                        self.log_reg_penalty = 'l1'
                    else:
                        logistic = linear_model.LogisticRegression(penalty="l2")#class_weight='auto'
                        logistic.fit(x_train, y_train)
                        logistic_predictions = logistic.predict(x_test)
                        if self.accuracy(logistic_predictions) > l1_accuracy:
                            self.log_reg_penalty = 'l2'
            if self.log_reg_C == None:
                max_accuracy = -1
                for C in xrange(1,104,4):
                    logistic = logistic = linear_model.LogisticRegression(penalty=self.log_reg_penalty, C=C)
                    logistic.fit(x_train, y_train)
                    logistic_predictions = logistic.predict(x_test)
                    if self.accuracy(logistic_predictions) > max_accuracy:
                        max_accuracy = self.accuracy(logistic_predictions)
                        self.log_reg_C = C
                print "Optimal Logistic Regression Hyperparams: Penalty = %s, C = %d."\
                    %(self.log_reg_penalty, self.log_reg_C)
            logistic = linear_model.LogisticRegression(penalty=self.log_reg_penalty, C = self.log_reg_C)#class_weight='auto'
            logistic.fit(x_train, y_train)
            logistic_predictions = logistic.predict(x_test)
            if self.stacking == True:
                return logistic_predictions, logistic.predict(self.stacking_test_data())
            return logistic_predictions
    
    def lda(self, x_train=None, y_train=None, x_test=None, y_test=None):
        x_train, y_train, x_test, y_test = self.initialize_data(x_train, y_train, x_test, y_test)
        lda = discriminant_analysis.LinearDiscriminantAnalysis()
        lda.fit(x_train, y_train)
        lda_predictions = lda.predict(x_test)
        if self.stacking == True:
            return lda_predictions, lda.predict(self.stacking_test_data())
        return lda_predictions
    
    def qda(self, x_train=None, y_train=None, x_test=None, y_test=None):
        x_train, y_train, x_test, y_test = self.initialize_data(x_train, y_train, x_test, y_test)
        if self.hyperparameters == 'default':
            qda = discriminant_analysis.QuadraticDiscriminantAnalysis()
            qda.fit(x_train, y_train)
            qda_predictions = qda.predict(x_test)
            return qda_predictions
        elif self.hyperparameters == 'individual':
            if self.qda_reg_param == None:
                max_accuracy = -1
                for reg_param in [x*.1 for x in range(0,10,2)]:
                    qda = discriminant_analysis.QuadraticDiscriminantAnalysis(reg_param = reg_param)
                    qda.fit(x_train, y_train)
                    qda_predictions = qda.predict(x_test)
                    if self.accuracy(qda_predictions) > max_accuracy:
                        max_accuracy = self.accuracy(qda_predictions)
                        self.qda_reg_param = reg_param
                print "Optimal QDA Hyperparams: Reg_param = %d" %(self.qda_reg_param)
            qda = discriminant_analysis.QuadraticDiscriminantAnalysis(reg_param = self.qda_reg_param)
            qda.fit(x_train, y_train)
            qda_predictions = qda.predict(x_test)
            if self.stacking == True:
                return qda_predictions, qda.predict(self.stacking_test_data())
            return qda_predictions
    
    def svc(self, x_train=None, y_train=None, x_test=None, y_test=None):
        x_train, y_train, x_test, y_test = self.initialize_data(x_train, y_train, x_test, y_test)
        if self.hyperparameters == 'default':
            support_vm = svm.SVC()
            support_vm.fit(x_train, y_train)
            svm_predictions = support_vm.predict(x_test)
            return svm_predictions
        elif self.hyperparameters == 'individual':
            if self.svc_C == None:
                max_accuracy = -1
                for C in [10, 100]:
                    s = svm.SVC(C=C)
                    s.fit(x_train, y_train)
                    svm_predictions = s.predict(x_test)
                    if self.accuracy(svm_predictions) > max_accuracy:
                        max_accuracy = self.accuracy(svm_predictions)
                        self.svc_C = C
            if self.svc_kernel == None:
                max_accuracy = -1
                for kernel in ['rbf', 'linear', 'poly']:
                    s = svm.SVC(C = self.svc_C, kernel=kernel)
                    s.fit(x_train, y_train)
                    svm_predictions = s.predict(x_test)
                    if self.accuracy(svm_predictions) > max_accuracy:
                        max_accuracy = self.accuracy(svm_predictions)
                        self.svc_kernel = kernel
                print "Optimal SVM Hyperparams: Kernel = %s, C = %d" %(self.svc_kernel, self.svc_C)
            s = svm.SVC(C = self.svc_C, kernel=self.svc_kernel)
            s.fit(x_train, y_train)
            svm_predictions = s.predict(x_test)
            if self.stacking == True:
                return svm_predictions, s.predict(self.stacking_test_data())
            return svm_predictions
    
    def blend(self, args):
        n = len(args)
        blended_predictions = []
        for i in xrange(len(args[0])):
            counts = {}
            for j in xrange(n):
                try:
                    counts[args[j][i]] += 1
                except KeyError:
                    counts[args[j][i]] = 1
            maximus = -float('inf')
            for index, value in counts.iteritems():
                if value > maximus:
                    max_value = index
                    maximus = value
            blended_predictions.append(max_value)
        return blended_predictions
    
    def run_algorithms(self):
        xgboost_predictions = self.xgboost()
        booster_predictions = self.gradient_booster()
        forest_predictions = self.random_forest()
        erf_predictions = self.extremely_random_forest()
        logistic_predictions = self.logistic_regression()
        lda_predictions = self.lda()
        qda_predictions = self.qda()
        return [xgboost_predictions, booster_predictions, forest_predictions, erf_predictions,\
                  logistic_predictions, lda_predictions, qda_predictions]
    
    def create_predictions(self):
        self.testing = True
        predictions = (self.run_algorithms() + self.stacker())
        self.testing = False
        
        predictions_df = pd.DataFrame(predictions).transpose()
        self.testing_df = predictions_df
        predictions_df.columns = ['xgboost_predictions','booster_predictions', 'forest_predictions', 'erf_predictions',\
                  'logistic_predictions', 'lda_predictions', 'qda_predictions', \
                   'xgboost_on_stack', 'booster_on_stack','forest_on_stack', \
                    'erf_on_stack', 'logistic_on_stack', 'lda_on_stack', 'qda_on_stack']
        
        optimal_predictions = self.blend([predictions_df[j].values for j in self.optimal_iterator])
        print "Time Elapsed = %d Seconds" %(time.time()-self.time)
        return optimal_predictions
    
    def predict(self):
        algorithms = self.run_algorithms()

        models = ['xgboost_predictions','booster_predictions', 'forest_predictions', 'erf_predictions',\
                  'logistic_predictions', 'lda_predictions', 'qda_predictions']
        for i, v in enumerate(algorithms):
        
            print self.accuracy(v), models[i]
        
        predictions = (algorithms + self.stacker())
        
        predictions_df = pd.DataFrame(predictions).transpose()
        self.training_df = predictions_df
        predictions_df.columns = ['xgboost_predictions','booster_predictions', 'forest_predictions', 'erf_predictions',\
                  'logistic_predictions', 'lda_predictions', 'qda_predictions', \
                   'xgboost_on_stack', 'booster_on_stack', \
                    'forest_on_stack', 'erf_on_stack', 'logistic_on_stack', 'lda_on_stack', \
                    'qda_on_stack']

        combinations = []
        col = predictions_df.columns[:-1]
        for i in xrange(1, len(col), 2):
            for j in itertools.combinations(col, i):
                combinations.append([k for k in j])

        maximus = -1
        for i in combinations:
            accuracy = self.accuracy(self.blend([predictions_df[j].values for j in i]))
            if accuracy > maximus:
                maximus = accuracy
                optimal_predictions = self.blend([predictions_df[j].values for j in i])
                self.optimal_iterator = i
                print str(accuracy) + str(i)
        
        if self.test != None: 
            return self.create_predictions()
        else:
            print "Time Elapsed = %d Seconds" %(time.time()-self.time)
            return optimal_predictions

In [303]:
iris = pd.read_csv(path + "iris.data", header=None)
y = iris[4]
iris = iris.drop([4], 1)
print "Iris:"

problem = classifier(iris,y, hyperparameters='individual', test = iris[:20])
predictions = problem.predict()

Iris:
Optimal Booster Hyperparams: n_estimators = 100, max_depth = 2
Optimal Forest Hyperparams: n_estimators = 100, max_features = 1
Optimal erf Hyperparams: n_estimators = 100, proportion = 1
Optimal Logistic Regression Hyperparams: Penalty = l1, C = 1.
Optimal QDA Hyperparams: Reg_param = 0
1.0 xgboost_predictions
1.0 booster_predictions
1.0 forest_predictions
1.0 erf_predictions
1.0 logistic_predictions
1.0 lda_predictions
1.0 qda_predictions
1.0['xgboost_predictions']
Time Elapsed = 18 Seconds


/Users/jnoxon/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:466: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [272]:
evaluate(hyperparameters='individual')

Iris:
Optimal Booster Hyperparams: n_estimators = 100, max_depth = 2
Optimal Forest Hyperparams: n_estimators = 100, max_features = 1
Optimal erf Hyperparams: n_estimators = 100, proportion = 1
Optimal Logistic Regression Hyperparams: Penalty = l1, C = 1.
Optimal QDA Hyperparams: Reg_param = 0
1.0 xgboost_predictions
1.0 booster_predictions
1.0 forest_predictions
1.0 erf_predictions
1.0 logistic_predictions
1.0 lda_predictions
1.0 qda_predictions
1.0['xgboost_predictions']

Wine Quality:


KeyboardInterrupt: 

In [258]:
evaluate(hyperparameters='individual')

Iris:
Optimal Booster Hyperparams: n_estimators = 100, max_depth = 2
Optimal Forest Hyperparams: n_estimators = 100, max_features = 1
Optimal erf Hyperparams: n_estimators = 100, proportion = 1
Optimal Logistic Regression Hyperparams: Penalty = l1, C = 1.
Optimal QDA Hyperparams: Reg_param = 0
1.0 xgboost_predictions
1.0 booster_predictions
1.0 forest_predictions
1.0 erf_predictions
1.0 logistic_predictions
1.0 lda_predictions
1.0 qda_predictions
Optimal Booster Hyperparams: n_estimators = 100, max_depth = 2
Optimal Forest Hyperparams: n_estimators = 100, max_features = 1
Optimal erf Hyperparams: n_estimators = 100, proportion = 1
Optimal Logistic Regression Hyperparams: Penalty = l1, C = 1.
Optimal QDA Hyperparams: Reg_param = 0
Optimal Booster Hyperparams: n_estimators = 100, max_depth = 2
Optimal Forest Hyperparams: n_estimators = 100, max_features = 4
Optimal erf Hyperparams: n_estimators = 100, proportion = 3
Optimal Logistic Regression Hyperparams: Penalty = l2, C = 1.
Optimal QD

In [211]:
import itertools
model_permutations = []
models = [problem.xgboost, problem.gradient_booster, problem.random_forest,\
          problem.extremely_random_forest, problem.logistic_regression, problem.lda, problem.qda]
testing = ['xgboost', 'grad_boost', 'rf', 'erf', 'lr', 'lda', 'qda']
for j in itertools.permutations(models, 2):
    model_permutations.append([k for k in j])

In [213]:
test = []
for j in itertools.permutations(testing, 2):
    test.append(j)

In [219]:
model_permutations[0]

[<bound method classifier.xgboost of <__main__.classifier instance at 0x10efc8a28>>,
 <bound method classifier.gradient_booster of <__main__.classifier instance at 0x10efc8a28>>]

In [241]:
df = pd.read_csv(path + 'Oracle Development/winequality-red.csv', sep=';')
Y = df['quality'].values
df = df.drop('quality',1)
print"\nWine Quality:"

problem = classifier(df, Y, hyperparameters='individual')
predictions = problem.stacker()
print [problem.accuracy(prediction) for prediction in predictions]


Wine Quality:
<bound method classifier.xgboost of <__main__.classifier instance at 0x10e4ac878>>
<bound method classifier.xgboost of <__main__.classifier instance at 0x10e4ac878>>
<bound method classifier.gradient_booster of <__main__.classifier instance at 0x10e4ac878>>
<bound method classifier.xgboost of <__main__.classifier instance at 0x10e4ac878>>
Optimal Booster Hyperparams: n_estimators = 100, max_depth = 8
<bound method classifier.random_forest of <__main__.classifier instance at 0x10e4ac878>>
<bound method classifier.xgboost of <__main__.classifier instance at 0x10e4ac878>>
Optimal Forest Hyperparams: n_estimators = 900, max_features = 4
<bound method classifier.extremely_random_forest of <__main__.classifier instance at 0x10e4ac878>>
<bound method classifier.xgboost of <__main__.classifier instance at 0x10e4ac878>>
Optimal erf Hyperparams: n_estimators = 100, proportion = 4
<bound method classifier.logistic_regression of <__main__.classifier instance at 0x10e4ac878>>
<bound 

In [175]:
df = pd.read_csv(path + 'Oracle Development/winequality-red.csv', sep=';')
Y = df['quality'].values
df = df.drop('quality',1)
print"\nWine Quality:"

problem = classifier(df, Y, hyperparameters='default')
predictions = problem.gradient_booster()
problem.accuracy(predictions)


Wine Quality:


0.653125

In [117]:
df = pd.read_csv(path + 'Oracle Development/winequality-red.csv', sep=';')
Y = df['quality'].values
df = df.drop('quality',1)
print"\nWine Quality:"

problem = classifier(df, Y, hyperparameters='individual')
predictions = problem.predict()


Wine Quality:
Optimal Booster Hyperparams: n_estimators = 700, max_depth = 5
Optimal Forest Hyperparams: n_estimators = 100, max_features = 3
Optimal erf Hyperparams: n_estimators = 100, proportion = 4
Optimal Logistic Regression Hyperparams: Penalty = l1, C = 1.
Optimal QDA Hyperparams: Reg_param = 0
0.625 xgboost_predictions
0.65625 booster_predictions
0.640625 forest_predictions
0.678125 erf_predictions
0.575 logistic_predictions
0.559375 lda_predictions
0.55625 qda_predictions
0.6625 blended_predictions


In [260]:
leads = pd.read_csv(path + 'Movoto/leads.csv')
one_hot_columns = ['First_Lead_Platform', 'State']
for i in one_hot_columns:
    dummies = pd.get_dummies(leads[i])
    for j in dummies:
        leads[j] = dummies[j]
y = leads['Met']
leads = leads.drop(['First_Lead_Platform', 'State', 'Met'], axis =1)
print"\nLeads Met:"

problem = classifier(leads, y, hyperparameters='individual')
predictions, optimal = problem.predict()


Leads Met:
Optimal Booster Hyperparams: n_estimators = 100, max_depth = 2
Optimal Forest Hyperparams: n_estimators = 100, max_features = 23
Optimal erf Hyperparams: n_estimators = 500, proportion = 23
Optimal Logistic Regression Hyperparams: Penalty = l1, C = 53.
Optimal QDA Hyperparams: Reg_param = 0
0.130831973899 xgboost_predictions
0.868515497553 booster_predictions
0.862316476346 forest_predictions
0.86101141925 erf_predictions
0.869004893964 logistic_predictions
0.869004893964 lda_predictions
0.544861337684 qda_predictions
Optimal Booster Hyperparams: n_estimators = 100, max_depth = 2
Optimal Forest Hyperparams: n_estimators = 900, max_features = 31
Optimal erf Hyperparams: n_estimators = 900, proportion = 23
Optimal Logistic Regression Hyperparams: Penalty = l1, C = 1.
Optimal QDA Hyperparams: Reg_param = 0
Optimal Booster Hyperparams: n_estimators = 100, max_depth = 2
Optimal Forest Hyperparams: n_estimators = 500, max_features = 40
Optimal erf Hyperparams: n_estimators = 500,

ValueError: too many values to unpack

In [ ]:
leads = pd.read_csv(path + 'Movoto/leads.csv')
one_hot_columns = ['First_Lead_Platform', 'State']
for i in one_hot_columns:
    dummies = pd.get_dummies(leads[i])
    for j in dummies:
        leads[j] = dummies[j]
y = leads['Met']
leads = leads.drop(['First_Lead_Platform', 'State', 'Met'], axis =1)
print"\nLeads Met:"

problem = classifier(leads, y, hyperparameters='individual')
predictions = problem.predict()

,xgboost_predictions,booster_predictions,forest_predictions,erf_predictions,logistic_predictions,lda_predictions,qda_predictions,blended_predictions,y_test
0,1,0,0,0,1,1,1,1,1
1,1,0,0,0,0,0,0,0,1
2,1,0,0,0,0,0,0,0,1
3,1,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0


In [124]:
# predictions_df.corr()

In [244]:
df = pd.read_csv(path + 'Oracle Development/winequality-red.csv', sep=';')
Y = df['quality'].values
df = df.drop('quality',1)
print"\nWine Quality:"

problem = classifier(df, Y, hyperparameters='individual')
predictions, y_test = problem.predict()

predictions_df = pd.DataFrame(predictions).transpose()
predictions_df.columns = ['xgboost_predictions','booster_predictions', 'forest_predictions', 'erf_predictions',\
                  'logistic_predictions', 'lda_predictions', 'qda_predictions', \
                  'blended_predictions', 'xgboost_on_stack', 'booster_on_stack', \
                    'forest_on_stack', 'erf_on_stack', 'logistic_on_stack', 'lda_on_stack', \
                    'qda_on_stack']

import itertools
permutations = []
col = predictions_df.columns[:-1]
for i in xrange(1, len(col), 2):
    for j in itertools.combinations(col, i):
        permutations.append([k for k in j])

maximus = -1
for i in permutations:
    accuracy = problem.accuracy(blend([predictions_df[j].values for j in i]))
    if accuracy >= maximus:
        maximus = accuracy
        print str(accuracy) + str(i)


Wine Quality:
Optimal Booster Hyperparams: n_estimators = 400, max_depth = 5
Optimal Forest Hyperparams: n_estimators = 900, max_features = 3
Optimal erf Hyperparams: n_estimators = 100, proportion = 4
Optimal Logistic Regression Hyperparams: Penalty = l1, C = 1.
Optimal QDA Hyperparams: Reg_param = 0
0.625 xgboost_predictions
0.6375 booster_predictions
0.653125 forest_predictions
0.659375 erf_predictions
0.575 logistic_predictions
0.559375 lda_predictions
0.55625 qda_predictions
0.659375 blended_predictions
Optimal Booster Hyperparams: n_estimators = 100, max_depth = 8
Optimal Forest Hyperparams: n_estimators = 900, max_features = 3
Optimal erf Hyperparams: n_estimators = 500, proportion = 4
Optimal Logistic Regression Hyperparams: Penalty = l2, C = 1.
Optimal QDA Hyperparams: Reg_param = 0
Optimal Booster Hyperparams: n_estimators = 100, max_depth = 5
Optimal Forest Hyperparams: n_estimators = 900, max_features = 9
Optimal erf Hyperparams: n_estimators = 500, proportion = 9
Optimal 

ValueError: Length mismatch: Expected axis has 15 elements, new values have 14 elements

In [247]:
predictions_df.columns = ['xgboost_predictions','booster_predictions', 'forest_predictions', 'erf_predictions',\
                  'logistic_predictions', 'lda_predictions', 'qda_predictions', \
                  'blended_predictions', 'xgboost_on_stack', 'booster_on_stack', \
                    'forest_on_stack', 'erf_on_stack', 'logistic_on_stack', 'lda_on_stack', \
                    'qda_on_stack']

import itertools
permutations = []
col = predictions_df.columns[:-1]
for i in xrange(1, len(col), 2):
    for j in itertools.combinations(col, i):
        permutations.append([k for k in j])

maximus = -1
for i in permutations:
    accuracy = problem.accuracy(blend([predictions_df[j].values for j in i]))
    if accuracy >= maximus:
        maximus = accuracy
        print str(accuracy) + str(i)

0.625['xgboost_predictions']
0.6375['booster_predictions']
0.653125['forest_predictions']
0.659375['erf_predictions']
0.659375['blended_predictions']
0.659375['xgboost_predictions', 'booster_predictions', 'qda_predictions']
0.6625['xgboost_predictions', 'forest_predictions', 'erf_predictions']
0.665625['xgboost_predictions', 'forest_predictions', 'logistic_predictions']
0.665625['xgboost_predictions', 'forest_predictions', 'qda_predictions']
0.665625['xgboost_predictions', 'erf_predictions', 'logistic_predictions']
0.66875['xgboost_predictions', 'erf_predictions', 'booster_on_stack']
0.66875['xgboost_predictions', 'blended_predictions', 'booster_on_stack']
0.678125['xgboost_predictions', 'blended_predictions', 'forest_on_stack']
0.678125['xgboost_predictions', 'booster_predictions', 'forest_predictions', 'erf_predictions', 'booster_on_stack', 'forest_on_stack', 'lda_on_stack']
0.678125['xgboost_predictions', 'booster_predictions', 'forest_predictions', 'erf_predictions', 'booster_on_st

In [109]:
permutations

[['xgboost_predictions'],
 ['booster_predictions'],
 ['forest_predictions'],
 ['erf_predictions'],
 ['logistic_predictions'],
 ['lda_predictions'],
 ['qda_predictions'],
 ['blended_predictions'],
 ['y_test'],
 ['xgboost_predictions', 'booster_predictions', 'forest_predictions'],
 ['xgboost_predictions', 'booster_predictions', 'erf_predictions'],
 ['xgboost_predictions', 'booster_predictions', 'logistic_predictions'],
 ['xgboost_predictions', 'booster_predictions', 'lda_predictions'],
 ['xgboost_predictions', 'booster_predictions', 'qda_predictions'],
 ['xgboost_predictions', 'booster_predictions', 'blended_predictions'],
 ['xgboost_predictions', 'booster_predictions', 'y_test'],
 ['xgboost_predictions', 'forest_predictions', 'erf_predictions'],
 ['xgboost_predictions', 'forest_predictions', 'logistic_predictions'],
 ['xgboost_predictions', 'forest_predictions', 'lda_predictions'],
 ['xgboost_predictions', 'forest_predictions', 'qda_predictions'],
 ['xgboost_predictions', 'forest_predic

In [73]:
array

[['xgboost_predictions'],
 ['booster_predictions'],
 ['forest_predictions'],
 ['erf_predictions'],
 ['logistic_predictions'],
 ['lda_predictions'],
 ['qda_predictions'],
 ['blended_predictions'],
 ['y_test'],
 ['xgboost_predictions', 'booster_predictions', 'forest_predictions'],
 ['xgboost_predictions', 'booster_predictions', 'erf_predictions'],
 ['xgboost_predictions', 'booster_predictions', 'logistic_predictions'],
 ['xgboost_predictions', 'booster_predictions', 'lda_predictions'],
 ['xgboost_predictions', 'booster_predictions', 'qda_predictions'],
 ['xgboost_predictions', 'booster_predictions', 'blended_predictions'],
 ['xgboost_predictions', 'booster_predictions', 'y_test'],
 ['xgboost_predictions', 'forest_predictions', 'erf_predictions'],
 ['xgboost_predictions', 'forest_predictions', 'logistic_predictions'],
 ['xgboost_predictions', 'forest_predictions', 'lda_predictions'],
 ['xgboost_predictions', 'forest_predictions', 'qda_predictions'],
 ['xgboost_predictions', 'forest_predic

In [42]:
import sklearn.metrics
metrics.confusion_matrix(predictions_df['booster_predictions'], predictions_df['y_test'])

array([[ 0,  0,  0,  1,  0,  0],
       [ 0,  2,  2,  2,  0,  0],
       [ 1,  6, 94, 26,  0,  0],
       [ 0,  2, 33, 93, 19,  3],
       [ 0,  0,  1,  9, 22,  2],
       [ 0,  0,  0,  1,  1,  0]])

In [49]:
flat_df = pd.DataFrame(columns = predictions_df.columns)
for i in predictions_df:
    print i, len(predictions_df[i])
    flat_df[i] = pd.get_dummies(predictions_df[i]).values.flatten()

xgboost_predictions 320
booster_predictions 320
forest_predictions 320


ValueError: Length of values does not match length of index

In [58]:
pd.get_dummies(predictions_df['erf_predictions'], columns=['0','1','2','3'], dummy_na=True)

,2.0,3.0,4.0,5.0,nan
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,0,1,0,0,0
5,1,0,0,0,0
6,1,0,0,0,0
7,1,0,0,0,0
8,0,1,0,0,0
9,0,1,0,0,0
